In [1]:
import os
import numpy as np
from PIL import Image

# Veri seti yolu
data_path = "/content/drive/MyDrive/gdrive/Fire-Detection"

# Sınıf etiketleri
class_names = ["0", "1"]

# Veri setindeki görüntü sayısı
num_samples = sum([len(files) for r, d, files in os.walk(data_path)])

# Veri seti boyutları
img_height, img_width = 128, 128
num_channels = 3

# X ve y için numpy dizileri
X = np.zeros((num_samples, img_height, img_width, num_channels), dtype=np.float32)
y = np.zeros((num_samples,), dtype=np.int32)

# Verileri yükleyip numpy dizilerine dönüştürme
index = 0
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path)
        img = img.resize((img_height, img_width))

        # Dört kanallı görüntüyü üç kanallıya dönüştürme
        img = img.convert("RGB")

        X[index] = np.array(img, dtype=np.float32) / 255.
        y[index] = i
        index += 1

# Veri setini karıştırma
np.random.seed(42)
shuffle_index = np.random.permutation(num_samples)
X, y = X[shuffle_index], y[shuffle_index]

# Veri setini train, test ve validation olarak ayırma
num_train_samples = int(0.7 * num_samples)
num_test_samples = int(0.2 * num_samples)
num_val_samples = num_samples - num_train_samples - num_test_samples

x_train, y_train = X[:num_train_samples], y[:num_train_samples]
x_test, y_test = X[num_train_samples:num_train_samples+num_test_samples], y[num_train_samples:num_train_samples+num_test_samples]
x_val, y_val = X[num_train_samples+num_test_samples:], y[num_train_samples+num_test_samples:]

# Dizileri npy dosyalarına kaydetme
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)
np.save('x_val.npy', x_val)
np.save('y_val.npy', y_val)


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [3]:
# Dizileri yükleme
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
x_val = np.load('x_val.npy')
y_val = np.load('y_val.npy')

In [4]:
# CNN Modelini Oluşturma
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, num_channels)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(1, activation='sigmoid'))

In [5]:
# Model derleme
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Model özeti
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 128)               7

In [7]:
# Modeli eğitme
history = cnn_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/100
15/15 [==============================] - 15s 72ms/step - loss: 0.9368 - accuracy: 0.7956 - val_loss: 0.3613 - val_accuracy: 0.8485
Epoch 2/100
15/15 [==============================] - 0s 25ms/step - loss: 0.3088 - accuracy: 0.8462 - val_loss: 0.3464 - val_accuracy: 0.8636
Epoch 3/100
15/15 [==============================] - 0s 24ms/step - loss: 0.2600 - accuracy: 0.8813 - val_loss: 0.3428 - val_accuracy: 0.8788
Epoch 4/100
15/15 [==============================] - 0s 24ms/step - loss: 0.1861 - accuracy: 0.9275 - val_loss: 0.3097 - val_accuracy: 0.8788
Epoch 5/100
15/15 [==============================] - 0s 22ms/step - loss: 0.1897 - accuracy: 0.9253 - val_loss: 0.4232 - val_accuracy: 0.8788
Epoch 6/100
15/15 [==============================] - 0s 20ms/step - loss: 0.1819 - accuracy: 0.9319 - val_loss: 0.4014 - val_accuracy: 0.9091
Epoch 7/100
15/15 [==============================] - 0s 19ms/step - loss: 0.1006 - accuracy: 0.9648 - val_loss: 0.3964 - val_accuracy: 0.9091
Epoch

In [8]:
test_loss, test_accuracy = cnn_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


5/5 [==============================] - 0s 7ms/step - loss: 0.9498 - accuracy: 0.8846
Test Loss: 0.9497626423835754
Test Accuracy: 0.8846153616905212


In [16]:
# Yeni görüntünün yolu
new_image_path = "/content/yangin.png"

# Görüntüyü yükleme ve işleme
new_image = Image.open(new_image_path)
new_image = new_image.resize((img_height, img_width))
new_image_array = np.array(new_image, dtype=np.float32) / 255.
new_image_array = np.expand_dims(new_image_array, axis=0)  # Tek örnek olduğunu belirtmek için boyut ekleyin

# Tahmin yapma
prediction = cnn_model.predict(new_image_array)

# Tahmin sonucunu yazdırma
if prediction > 0.5:
    print("Görüntü yangın içeriyor.")
else:
    print("Görüntüde yangın yok.")

1/1 [==============================] - 0s 28ms/step
Görüntü yangın içeriyor.
